In [4]:
import os
from dotenv import load_dotenv
load_dotenv() #load environment variables from .env file

True

# Connect to database using ORM 

In [7]:
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
youtube = googleapiclient.discovery.build(serviceName='youtube', version='v3', developerKey=GOOGLE_API_KEY)

In [1]:
from dataclasses import dataclass, asdict
import json
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String, Integer, Boolean, DateTime
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
import datetime

POSTGRE_USER = os.environ.get("POSTGRE_USER")
POSTGRE_PASSWORD = os.environ.get("POSTGRE_PASSWORD")
POSTGRE_HOST = os.environ.get("POSTGRE_HOST")
POSTGRE_PORT = os.environ.get("POSTGRE_PORT")
engine = create_engine(f'postgresql+psycopg2://{POSTGRE_USER}:{POSTGRE_PASSWORD}@{POSTGRE_HOST}:{POSTGRE_PORT}/youtube', echo=True)

class Base(DeclarativeBase):
    pass

@dataclass
class YoutubeSearch(Base): 
    __tablename__ = "search"

    #technical id
    #TODO how to autoincrement id?
    id: Mapped[int] = mapped_column(primary_key=True) 

    #https://developers.google.com/youtube/v3/docs/search/list?hl=de
    #input parameter
    #mandatory input parameter
    part: Mapped[str] = mapped_column(String(30))
    #optionl input parameter
    forContentOwner: Mapped[Optional[bool]] = mapped_column(Boolean)
    forDeveloper: Mapped[Optional[bool]] = mapped_column(Boolean)
    forMine: Mapped[Optional[bool]] = mapped_column(Boolean)
    relatedToVideoId: Mapped[Optional[str]] = mapped_column(String(30))
    channelId: Mapped[Optional[str]] = mapped_column(String(30))
    channelType: Mapped[Optional[str]] = mapped_column(String(30))
    eventType: Mapped[Optional[str]] = mapped_column(String(30))
    location: Mapped[Optional[str]] = mapped_column(String(30))
    locationRadius: Mapped[Optional[str]] = mapped_column(String(30))
    maxResults: Mapped[Optional[int]] = mapped_column(Integer)
    onBehalfOfContentOwner: Mapped[Optional[str]] = mapped_column(String(30))
    order: Mapped[Optional[str]] = mapped_column(String(30))
    pageToken: Mapped[Optional[str]] = mapped_column(String(30))
    publishedAfter: Mapped[Optional[datetime.datetime]] = mapped_column(DateTime(timezone=True))
    publishedBefore: Mapped[Optional[datetime.datetime]] = mapped_column(DateTime(timezone=True))
    q: Mapped[Optional[str]] = mapped_column(String(30))
    regionCode: Mapped[Optional[str]] = mapped_column(String(2))
    relevanceLanguage: Mapped[Optional[str]] = mapped_column(String(2))
    safeSearch: Mapped[Optional[str]] = mapped_column(String(30))
    topicId: Mapped[Optional[str]] = mapped_column(String(30))
    type: Mapped[Optional[str]] = mapped_column(String(30))
    videoCaption: Mapped[Optional[str]] = mapped_column(String(30))
    videoCategoryId: Mapped[Optional[str]] = mapped_column(String(30))
    videoDefinition: Mapped[Optional[str]] = mapped_column(String(30))
    videoDimension: Mapped[Optional[str]] = mapped_column(String(30))
    videoDuration: Mapped[Optional[str]] = mapped_column(String(30))
    videoEmbeddable: Mapped[Optional[str]] = mapped_column(String(30))
    videoLicense: Mapped[Optional[str]] = mapped_column(String(30))
    videoSyndicated: Mapped[Optional[str]] = mapped_column(String(30))
    videoType: Mapped[Optional[str]] = mapped_column(String(30))

    #output parameter
    response: Mapped[Optional[List["YoutubeResponse"]]] = relationship(back_populates="YoutubeSearch", cascade="all, delete-orphan")

    def __init__(self, part, forContentOwner=None, forDeveloper=None, forMine=None, relatedToVideoId=None, channelId=None, channelType=None, eventType=None, location=None, locationRadius=None, maxResults=None, onBehalfOfContentOwner=None, order=None, pageToken=None, publishedAfter=None, publishedBefore=None, q=None, regionCode=None, relevanceLanguage=None, safeSearch=None, topicId=None, type=None, videoCaption=None, videoCategoryId=None, videoDefinition=None, videoDimension=None, videoDuration=None, videoEmbeddable=None, videoLicense=None, videoSyndicated=None,  videoType=None):
        #TODO autoincrement id
        self.part = part
        self.forContentOwner = forContentOwner
        self.forDeveloper = forDeveloper
        self.forMine = forMine
        self.relatedToVideoId = relatedToVideoId
        self.channelId = channelId
        self.channelType = channelType
        self.eventType = eventType
        self.location = location
        self.locationRadius = locationRadius
        self.maxResults = maxResults
        self.onBehalfOfContentOwner = onBehalfOfContentOwner
        self.order = order
        self.pageToken = pageToken
        self.publishedAfter = publishedAfter
        self.publishedBefore = publishedBefore
        self.q = q
        self.regionCode = regionCode
        self.relevanceLanguage = relevanceLanguage
        self.safeSearch = safeSearch
        self.topicId = topicId
        self.type = type
        self.videoCaption = videoCaption
        self.videoCategoryId = videoCategoryId
        self.videoDefinition = videoDefinition
        self.videoDimension = videoDimension
        self.videoDuration = videoDuration
        self.videoEmbeddable = videoEmbeddable
        self.videoLicense = videoLicense
        self.videoSyndicated = videoSyndicated
        self.videoType = videoType
    
    #https://www.youtube.com/watch?v=Fu0swCLAJ8E
    def execute(self):
        self_attrs = vars(self)
        not_none_params = {k:v for k, v in self_attrs.items() if v is not None}
        del not_none_params["id"]
        del not_none_params["response"]
        request = youtube.serach().list(**not_none_params)
        response = request.execute()
        #TODO parse JSON response to object 
        self.response = YoutubeResponse(**response)
        #TODO persit query
        self.persist()
    
    def persist(self):
        session = Session(engine)
        session.add_all(self)
        session.commit()


@dataclass(frozen=True, order=True)
class YoutubeResponse(Base):
    __tablename__ = "response"

    #technical id
    #TODO autoincrement
    id: Mapped[int] = mapped_column(primary_key=True)

    #https://www.youtube.com/watch?v=Fu0swCLAJ8E
    kind: Mapped[str] = mapped_column(String(30))
    etag: Mapped[str] = mapped_column(String(30))
    nextPageToken: Mapped[str] = mapped_column(String(30))
    regionCode: Mapped[str] = mapped_column(String(2))
    pageInfo: Mapped[List["YoutubePageInfo"]] = relationship(back_populates="YoutubeResponse", cascade="all, delete-orphan")

    def __post_init__(self):
        self.pageInfo = [YoutubePageInfo(**pageInfo) for pageInfo in self.pageInfo]

@dataclass(frozen=True, order=True)
class YoutubePageInfo(Base):
    __tablename__ = "page_info"

    #technical id
    #TODO autoincrement
    id: Mapped[int] = mapped_column(primary_key=True)

    #output parameter
    totalResults: Mapped[int] = mapped_column(Integer)
    resultsPerPage: Mapped[int] = mapped_column(Integer)



@dataclass(frozen=True, order=True)
class YoutubeSearchResouce(Base):
    __tablename__ = "search_resource"

    #technical id
    #TODO autoincrement
    id: Mapped[int] = mapped_column(primary_key=True)




# Connect to youtube API

In [15]:
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
youtube = googleapiclient.discovery.build(serviceName='youtube', version='v3', developerKey=GOOGLE_API_KEY)

def youtubeSearch(q):
    request = youtube.search().list(
        part = "snippet",
        eventType = "completed",
        q = q,
        type = "video"
    )
    response = request.execute()
    return response

# Check if youtube query string exists

In [ ]:
import psycopg2
POSTGRE_USER = os.environ.get("POSTGRE_USER")
POSTGRE_PASSWORD = os.environ.get("POSTGRE_PASSWORD")
POSTGRE_HOST = os.environ.get("POSTGRE_HOST")
conn = psycopg2.connect(dbname="youtube_search", user=POSTGRE_USER, password=POSTGRE_PASSWORD, host=POSTGRE_HOST)
cur = conn.cursor()

# Execute Youtube querry

In [16]:
import json

response = youtubeSearch("airport ambient noise")
#response_json= json.loads(json.dumps(response))
print(response)

{'kind': 'youtube#searchListResponse', 'etag': 'swqIWhX5YPhnqhYhfIHUZGK3kuE', 'nextPageToken': 'CAUQAA', 'regionCode': 'DE', 'pageInfo': {'totalResults': 350345, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#searchResult', 'etag': 'tIFIp7Wt1NK6nBtWTaw6bNsTyfU', 'id': {'kind': 'youtube#video', 'videoId': 'MjPWVfHAEfw'}, 'snippet': {'publishedAt': '2021-11-16T05:34:34Z', 'channelId': 'UCbK6VFemFGeVlC2vm9DpYLQ', 'title': 'NEW Airport Background Ambience - Terminal,Announcement, Boarding Sound Effects', 'description': 'Did you miss travelling due COVID-19? This is Real Complete Airport Ambience Sounds Effects FOR 1 HOUR which includes: ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/MjPWVfHAEfw/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/MjPWVfHAEfw/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/MjPWVfHAEfw/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'AMBIENCE BOOSTER', 'l

# download Video, extract audio, save audio file on Server